# Building Custom Ansible Modules in Python

Have you ever wanted to type

`ansible -m my-really-cool-custom-module -a 'param1=foo param2=bar'`

... and have it actually work?

Then this section may be for you!

(much of this material is taken from Redhat's [Ansible Module Development Walkthrough][1])

[1]: http://docs.ansible.com/ansible/latest/dev_guide/developing_modules_general.html

*Don't* create a custom module until

- you've looked through all the built-in Ansible modules and can't find something that does what you want
- you've decided that using the `raw` and `script` modules is not powerful enough for what you need
- you know some Python

# Environment setup

- Clone the Ansible repository: `$ git clone https://github.com/ansible/ansible.git`
- Change directory into the repository root dir: `$ cd ansible`
- Create a virtual environment: `$ python3 -m venv venv` (or for Python 2 $ `virtualenv venv`. Note, this requires you to install the virtualenv package: `$ pip install virtualenv`)
- Activate the virtual environment: `$ . venv/bin/activate`
- Install development requirements: `$ pip install -r requirements.txt`
- Run the environment setup script for each new dev shell process: `$ . hacking/env-setup`



In [3]:
cd ~/src/Classes/Ansible

/Users/rick446/src/Classes/Ansible


In [4]:
%%bash
git clone https://github.com/ansible/ansible.git
cd ansible
python3 -m venv venv
. venv/bin/activate
pip install -r requirements.txt
. hacking/env-setup

Process is interrupted.


A minimal module can be found in `modules/simple_module.py` (a commented version is available in `modules/my_new_test_module.py`):

In [5]:
cat modules/les/simple_module.py

from ansible.module_utils.basic import AnsibleModule

def run_module():
    module_args = dict(
        name=dict(type='str', required=True),
        new=dict(type='bool', required=False, default=False)
    )
    result = dict(
        changed=False,
        original_message='',
        message=''
    )
    module = AnsibleModule(
        argument_spec=module_args,
        supports_check_mode=True
    )
    if module.check_mode:
        return result
    result['original_message'] = module.params['name']
    result['message'] = 'goodbye'
    if module.params['new']:
        result['changed'] = True
    if module.params['name'] == 'fail me':
        module.fail_json(msg='You requested this to fail', **result)
    module.exit_json(**result)


if __name__ == '__main__':
    run_module()


In [6]:
%%bash
ansible localhost -m simple_module

localhost | FAILED! => {
    "changed": false,
    "failed": true,
    "msg": "missing required arguments: name"
}


In [7]:
%%bash
ansible localhost -m simple_module -a name=foo

localhost | SUCCESS => {
    "changed": false,
    "failed": false,
    "message": "goodbye",
    "original_message": "foo"
}


In [8]:
%%bash
ansible localhost -m simple_module -a 'name="fail me"'

localhost | FAILED! => {
    "changed": false,
    "failed": true,
    "message": "goodbye",
    "msg": "You requested this to fail",
    "original_message": "fail me"
}


In [9]:
%%bash
ansible localhost -m simple_module -a 'name=foo new=true'

localhost | SUCCESS => {
    "changed": true,
    "failed": false,
    "message": "goodbye",
    "original_message": "foo"
}


# Lab: Modify your module

Modify the module to report on the number of files in the user's home directory (hint: use `os.listdir()` and `os.environ['HOME']`. Confirm that it works locally.